## Where to find the data

The data for the bias current sweeps for the two tunable resonators is in:

```
IRdetection/Experiments/TRswipeAuto/
```

In the `run-0/data` folder is present the data for the higher frequency resonator, and in the `run-2/data` folder is present the data for the lower frequency resonator. (run-1 crashed).

The data for all the sweep is saved as a *hdf5* file.

Morehover in the `run-i/figures` folder are the plots of the amplitude at each bias current.

## Data structure
The HDF5 data files are structured as follows:

```plaintext
# tr_sweep_data.h5

┌───────────────────────────────────────────────────────┐
│ 📊 Metadata       
                                   │
├───────────────────────────────────────────────────────┤
│ ├─ attenuation (dBm)          :                       │
│ ├─ cryostat_temperature (K)   :                       │
│ ├─ f0 (Hz)                    :                       │
│ ├─ frequency_span (Hz)        :                       │
│ ├─ param_a                    :                       │
│ ├─ param_b                    :                       │
│ ├─ resistance (Ohm)           :                       │
│ ├─ vna_avg_count              :                       │
│ ├─ vna_bandwidth (Hz)         :                       │
│ ├─ vna_point_count            :                       │
│ ├─ vna_power (dBm)            :                       │
│ ├─ voltage_max (V)            :                       │
│ ├─ voltage_min (V)            :                       │
│ └─ voltage_step (V)           :                       │
│                                                       │
├───────────────────────────────────────────────────────┤
│ 📈 voltage_sweeps                                     │
├───────────────────────────────────────────────────────┤
│ ├─ bias_0.000V                                        │
│ │  ├─ Data                                            │
│ │  │  ├─ column 0             : frequency (Hz)        │
│ │  │  ├─ column 1             : real                  │
│ │  │  └─ column 2             : imag                  │
│ │  │                                                  │
│ │  └─ Metadata                                        │
│ │     ├─ bias_current_mA                              │
│ │     ├─ bias_voltage                                 │
│ │     ├─ center_frequency     : [expected peak freq]  │
│ │     └─ column_names                                 │
│ │                                                     │
│ ├─ bias_0.200V                                        │
│ │  └─ ...                                             │
│ │                                                     │
│ └─ ... and so on for each voltage step                │
└───────────────────────────────────────────────────────┘
```

Each voltage sweep contains measurements at different bias voltages, starting from 0.000V and incremented by the voltage_step value (0.2V) up to voltage_max (8V).

### IMPORTANT NOTE!

The resistance value saved in the datafiles is incorrect. We have later measured it better, it turns out to be: 
$R=2.317 \, k\Omega$

Because of this, the bias current must be recomputed and the ones saved in the metadatas should not be considered. In the data class below this is already handled.

## Reading the data

In [3]:
import h5py
import numpy as np
import os
import pandas as pd 
from dataclasses import dataclass

In [ ]:
@dataclass
class SweepData:
    filepath: str
    metadata: dict = None  # Global metadata
    sweep_voltages: list = None  # List of voltage values
    sweeps: dict = None  # Dictionary of sweep data by voltage
    
    def __post_init__(self):
        self.metadata = {}
        self.sweeps = {}
        self.sweep_voltages = []
        self._load_data()
    
    def _load_data(self):
        """Load data from HDF5 file"""
        with h5py.File(self.filepath, 'r') as f:
            # Load global metadata
            for key in f.attrs.keys():
                self.metadata[key] = f.attrs[key]
            
            # Correct resistance
            self.metadata['resistance'] = 2.317e3
            
            # Load voltage sweeps
            if 'voltage_sweeps' in f:
                sweep_group = f['voltage_sweeps']
                for voltage_key in sweep_group.keys():
                    # Extract voltage value from key (bias_X.XXXV)
                    voltage_str = voltage_key.split('_')[1].split('V')[0]
                    voltage = float(voltage_str)
                    self.sweep_voltages.append(voltage)
                    
                    sweep = sweep_group[voltage_key]
                    
                    # Create a dictionary for this sweep
                    sweep_data = {
                        'data': {},
                        'metadata': {}
                    }
                    
                    # Load sweep metadata
                    if sweep.attrs is not None:
                        meta_group = sweep.attrs
                        for key in meta_group.keys():
                            sweep_data['metadata'][key] = meta_group.get(key, None)
                        
                        # Correct bias current
                        sweep_data['metadata']['bias_current_mA'] = sweep_data['metadata'].get('bias_voltage') * 1e3 / self.metadata['resistance']
                    
                    # Load data columns
                    data_group = sweep[()]
                    column_names = []
                    
                    # Get column names if available in metadata
                    if 'column_names' in sweep_data['metadata']:
                        column_names = sweep_data['metadata']['column_names']
                    
                    # Load each column
                    for i in range(len(data_group)):
                        # Use proper column name if available, otherwise use default name
                        if i < len(column_names) and column_names[i]:
                            col_name = column_names[i]
                        else:
                            col_name = f'column_{i}'
                        
                        # Load the data directly, h5py handles the conversion to numpy array
                        sweep_data['data'][col_name] = data_group[i]
                    
                    self.sweeps[voltage] = sweep_data
            
        # Sort voltage list for easier iteration
        self.sweep_voltages = sorted(self.sweep_voltages)
    
    def get_sweep(self, voltage):
        """Get data for a specific voltage"""
        return self.sweeps.get(voltage)
    
    def get_frequency_data(self, voltage):
        """Get frequency, real, and imaginary data for a specific voltage"""
        sweep = self.get_sweep(voltage)
        if sweep is None:
            return None, None, None
        
        # Try standard names first, then fall back to column positions
        freq = sweep['data'].get('frequency (Hz)', 
                                sweep['data'].get('frequency', 
                                                 sweep['data'].get('column_0')))
        real = sweep['data'].get('real', sweep['data'].get('column_1'))
        imag = sweep['data'].get('imag', sweep['data'].get('column_2'))
        
        return freq, real, imag
    
    def get_amplitude(self, voltage):
        """Calculate amplitude from real and imaginary parts"""
        freq, real, imag = self.get_frequency_data(voltage)
        if freq is None:
            return None, None
        
        amplitude = np.sqrt(real**2 + imag**2)
        return freq, amplitude
    
    def get_phase(self, voltage):
        """Calculate phase from real and imaginary parts"""
        freq, real, imag = self.get_frequency_data(voltage)
        if freq is None:
            return None, None
        
        phase = np.arctan2(imag, real)
        return freq, phase
    
    def to_dataframe(self, voltage):
        """Convert sweep data to pandas DataFrame"""
        freq, real, imag = self.get_frequency_data(voltage)
        if freq is None:
            return None
        
        df = pd.DataFrame({
            'frequency': freq,
            'real': real,
            'imag': imag,
            'amplitude': np.sqrt(real**2 + imag**2),
            'phase': np.arctan2(imag, real)
        })
        return df

## Example usage of SweepData class

In [39]:
# Path to the high frequency resonator data file
high_freq_path = "../Experiments/TRswipeAuto/run-0/data/tr_sweep_data.h5"

# Check if file exists
if os.path.exists(high_freq_path):
    # Load the data
    high_freq_data = SweepData(high_freq_path)
    
    # Print global metadata
    print("Global Metadata:")
    for key, value in high_freq_data.metadata.items():
        print(f"  {key}: {value}")
    
    # Print available voltage sweeps
    print(f"\nAvailable voltage sweeps: {high_freq_data.sweep_voltages}")
    
    # Get data for the first voltage
    if high_freq_data.sweep_voltages:
        first_voltage = high_freq_data.sweep_voltages[1]
        sweep_metadata = high_freq_data.get_sweep(first_voltage)['metadata']
        
        print(f"\nMetadata for {first_voltage}V sweep:")
        for key, value in sweep_metadata.items():
            print(f"  {key}: {value}")
        
        # Convert to DataFrame
        df = high_freq_data.to_dataframe(first_voltage)
        if df is not None:
            print(f"\nDataFrame for {first_voltage}V sweep:")
            print(df.head())
else:
    print(f"File not found: {high_freq_path}")

Global Metadata:
  attenuation (dBm): -20
  cryostat_temperature (K): 1.4
  f0 (Hz): 6234427256
  frequency_span (Hz): 7500000.0
  param_a: 12.6
  param_b: 9.28
  resistance (Ohm): 1997.0
  vna_avg_count: 5
  vna_bandwidth (Hz): 1000
  vna_point_count: 1000
  vna_power (dBm): -30
  voltage_max (V): 8.0
  voltage_min (V): 0.0
  voltage_step (V): 0.2
  resistance: 2317.0

Available voltage sweeps: [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0, 4.2, 4.4, 4.6, 4.8, 5.0, 5.2, 5.4, 5.6, 5.8, 6.0, 6.2, 6.4, 6.6, 6.8, 7.0, 7.2, 7.4, 7.6, 7.8, 8.0]

Metadata for 0.2V sweep:
  bias_current_mA: 0.100150225338007
  bias_voltage: 0.2
  center_frequency: 6234230000.0
  column_names: ['frequency' 'real' 'imag']
  bias_current: 8.631851532153647e-05

DataFrame for 0.2V sweep:
      frequency      real      imag  amplitude     phase
0  6.230480e+09  0.005751  0.039810   0.040223  1.427331
1  6.230488e+09  0.006579  0.039450   0.039995  1.405548